In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.extra.rate_limiter import RateLimiter
import time 

df = pd.read_csv('../uncleaned_datasets/data_fnb_bandung.csv')

df = df[df['tahun'] == 2024]


df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
Index: 3080 entries, 6041 to 9120
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       3080 non-null   int64 
 1   kode_provinsi            3080 non-null   int64 
 2   nama_provinsi            3080 non-null   object
 3   bps_kode_kabupaten_kota  3080 non-null   int64 
 4   bps_nama_kabupaten_kota  3080 non-null   object
 5   nama_rumah_makan         3080 non-null   object
 6   alamat                   3080 non-null   object
 7   tahun                    3080 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 216.6+ KB


id                          int64
kode_provinsi               int64
nama_provinsi              object
bps_kode_kabupaten_kota     int64
bps_nama_kabupaten_kota    object
nama_rumah_makan           object
alamat                     object
tahun                       int64
dtype: object

In [6]:
# df.head()

In [7]:
def get_area_details_by_place_and_street(street_name):
    """
    Mendapatkan detail kelurahan dan kecamatan berdasarkan nama tempat dan nama jalan.
    """
    # Penting: Ganti 'my_combined_search_app' dengan nama aplikasi Anda yang unik
    geolocator = Nominatim(user_agent="my_combined_search_app")

    # Gabungkan nama tempat dan nama jalan untuk membuat alamat yang lebih spesifik
    # Anda bisa menambahkan kota/provinsi/negara jika dirasa perlu untuk akurasi yang lebih tinggi
    full_address = f"{street_name}, Bandung, West Java, Indonesia"

    try:
        # Lakukan geocoding
        # addressdetails=True sangat penting untuk mendapatkan detail komponen alamat
        location = geolocator.geocode(full_address, addressdetails=True, language='id', timeout=10)

        if location:
            address_components = location.raw.get('address', {})
            
            # Mendapatkan Kelurahan/Desa
            # Cek berbagai kemungkinan kunci yang digunakan Nominatim
            kelurahan = address_components.get('village') or \
                        address_components.get('suburb') or \
                        address_components.get('locality') or \
                        address_components.get('hamlet') # 'hamlet' bisa jadi opsi lain untuk desa kecil

            # Mendapatkan Kecamatan
            # Cek berbagai kemungkinan kunci yang digunakan Nominatim
            kecamatan = address_components.get('county') or \
                        address_components.get('state_district') or \
                        address_components.get('city_district') or \
                        address_components.get('town')

            # print(f"Pencarian untuk: '{street_name}' di '{place_name}'")
            print(f"Alamat Lengkap Ditemukan: {location.address}")
            print(f"Latitude: {location.latitude}")
            print(f"Longitude: {location.longitude}")
            print(f"Kelurahan/Desa: {kelurahan if kelurahan else 'Tidak ditemukan'}")
            print(f"Kecamatan: {kecamatan if kecamatan else 'Tidak ditemukan'}")
            print("-" * 40)

            data = [kelurahan, kecamatan, location.latitude, location.longitude]
            return pd.Series(data)

        else:
            print(f"Tidak dapat menemukan detail lokasi untuk: '{street_name}'")
            print("-" * 40)
            return pd.Series([None, None, None, None])

    except GeocoderTimedOut:
        print("Error: Layanan geocoding time out. Silakan coba lagi nanti.")
        return pd.Series([None, None, None, None])
    except GeocoderServiceError as e:
        print(f"Error dari layanan geocoding: {e}")
        return pd.Series([None, None, None, None])
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return pd.Series([None, None, None, None])

In [8]:
df[['kelurahan', 'kecamatan', 'latitude', 'longitude']] = df.apply(
    lambda row: get_area_details_by_place_and_street(row['alamat']),
    axis=1
)

df.head()

# Simpan hasil ke file CSV baru
df.to_csv('../datasets/data_fnb_bandung_cleaned.csv', index=False)
print("Data telah disimpan ke 'data_fnb_bandung_cleaned.csv'.")

Tidak dapat menemukan detail lokasi untuk: '3HX7+GPJ, BANDUNG, CAMPAKA, KEC. ANDIR, KOTA BANDUNG, JAWA
  BARAT 40184'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'JL. SAPTA MARGA NO.72'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: '4H38+J9F, JL. BABAKAN CIANJUR'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: '4H28+J98'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'JL. RAYA CIBEUREUM GG. RH. JEJENG NO.140'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'CAMPAKA, KEC. ANDIR, KOTA BANDUNG, JAWA BARAT 40175'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: '3HV9+PVV, JL. BABAKAN CIBEUREUM'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'JL. BABAKAN CIBEUREUM'
----------------------------------------
Tidak dapat menemukan detail lo